<a href="https://colab.research.google.com/github/LuigiSigillo/nlp2021-hw/blob/master/hw1/stud/nlp_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [1]:
from google.colab import drive
# general
import matplotlib.pyplot as plt
import numpy as np
import os
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
from typing import *

# torch
import torch
import json
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD

torch.manual_seed(42)

drive.mount('/content/drive')
root_folder = '/content/drive/My Drive/NLP/nlp2021-hw1'
dataset_folder = os.path.join(root_folder,'data')
print(dataset_folder)

Mounted at /content/drive
/content/drive/My Drive/NLP/nlp2021-hw1/data


In [ ]:
#! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
#! unzip -d data/glove.6B
#! cd '/content/drive/My Drive/NLP/nlp2021-hw1'
#!unzip '/content/drive/My Drive/NLP/nlp2021-hw1/glove.6B.zip'
!mv glove.6B.200d.txt '/content/drive/My Drive/NLP/nlp2021-hw1/model'
!ls

 load the actual word embeddings

In [2]:
word_vectors = dict()
words_limit = 100_000
with open('/content/drive/My Drive/NLP/nlp2021-hw1/model/glove.6B.100d.txt') as f:

    next(f)  # skip header

    for i, line in tqdm(enumerate(f), total=words_limit):

        if i == words_limit:
            break

        word, *vector = line.strip().split(' ')
        vector = torch.tensor([float(c) for c in vector])
        
        word_vectors[word] = vector

## alternatives
# for l in f:
#     line = l.decode().split()
#     word = line[0]
#     words.append(word)
#     word2idx[word] = idx
#     idx += 1
#     vect = np.array(line[1:]).astype(np.float)
#     vectors.append(vect)


In [3]:
def cosine_similarity(v1: torch.Tensor, v2: torch.Tensor) -> float:
    num = torch.sum(v1 * v2)
    den = torch.linalg.norm(v1) * torch.linalg.norm(v2)
    return (num / den).item()

In [4]:
cosine_similarity(word_vectors['king'], word_vectors['queen'])

0.7507690787315369

word-embedding-powered function $\phi$.  just converts any review to a vector by **averaging the embeddings corresponding to each word in it**.

In [3]:
def phrase2vector(phrase: str) -> Optional[torch.Tensor]:
    phrases_word_vector = [word_vectors[w] for w in phrase.split(' ') if w in word_vectors]
    
    if len(phrases_word_vector) == 0:
        return None

    phrases_word_vector = torch.stack(phrases_word_vector)  # tensor shape: (#words X #features)
    return torch.mean(phrases_word_vector, dim=0)

In [4]:
class SentencesDataset(torch.utils.data.Dataset):

    def __init__(self, dataset_path: str, phrase2vector):
        self.data_store = []
        self.init_structures(dataset_path, phrase2vector)

    def init_structures(self, dataset_path: str, phrase2vector) -> None:

        with open(dataset_path) as f:
            for json_string in f:
                single_json = json.loads(json_string)
                '''s1 = single_json['sentence1']
                s2 =  single_json['sentence2']
                ground_t = 1 if single_json['label'] =='True' else 0
                vector1 = phrase2vector(s1)
                vector2 = phrase2vector(s2)
                if vector1 is None:
                    continue
                if vector2 is None:
                    continue
                self.data_store.append((vector1,vector2,ground_t))'''
                sentence =  single_json['sentence1'] + " <SEP> " + single_json['sentence2']
                ground_t = np.float32(1) if single_json['label'] =='True' else np.float32(0)
                vector = phrase2vector(sentence)
                if vector is None:
                    continue
                self.data_store.append((vector,ground_t))


    def __len__(self) -> int:
        return len(self.data_store)

    def __getitem__(self, idx: int) -> torch.Tensor:
        return self.data_store[idx]

In [5]:
class SentencesDataModule(nn.Module):

    def __init__(
        self, 
        data_train_path: str,
        data_dev_path: str,
        data_test_path: str,
        batch_size: int,
        collate_fn=None
    ) -> None:
        super().__init__()
        self.data_train_path = data_train_path
        self.data_dev_path = data_dev_path
        self.data_test_path = data_test_path
        self.batch_size = batch_size
        self.collate_fn = collate_fn

        self.train_dataset = None
        self.validation_dataset = None
        self.test_dataset = None

    def setup(self, stage: Optional[str] = None) -> None:
        if stage == 'fit':
            self.train_dataset = SentencesDataset(self.data_train_path, phrase2vector)
            self.validation_dataset = SentencesDataset(self.data_dev_path, phrase2vector)
        elif stage == 'test':
            self.test_dataset = SentencesDataset(self.data_test_path, phrase2vector)

    def train_dataloader(self, *args, **kwargs) -> DataLoader:
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(self.validation_dataset, batch_size=self.batch_size)

    def test_dataloader(self, *args, **kwargs) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [6]:
sentences_dm = SentencesDataModule(
    data_train_path=dataset_folder+'/train.jsonl',
    data_dev_path=dataset_folder+'/dev.jsonl',
    data_test_path=dataset_folder+'/dev.jsonl',
    batch_size=32,
)
sentences_dm.setup('test')
test_dataloader = sentences_dm.test_dataloader()
# print(word_vectors['test'])

for batch in test_dataloader:
    X, y = batch
    print(batch)
    print(f"batch X shape: {X.shape}")
    print(f"batch z shape: {y.shape}")
    
    
    break

# Non so se training

In [8]:
class SentencesClassifier(nn.Module):

    def __init__(self, n_features: int, n_hidden: int):
        super().__init__()
        # classification function
        self.lin1 = torch.nn.Linear(n_features, n_hidden)
        self.output_layer = torch.nn.Linear(n_hidden, 1)
        
        # criterion
        self.loss_fn = torch.nn.BCELoss()
        

    def forward(self, x: torch.Tensor, y: Optional[torch.Tensor] = None) -> Dict[str, torch.Tensor]:
        # actual forward
        out = self.lin1(x)
        out = torch.relu(out)
        # compute logits (which are simply the out variable) and the actual probability distribution (pred, as it is the predicted distribution)
    
        logits = self.output_layer(out).squeeze(1)

        out = torch.sigmoid(logits)

        result = {'logits': logits, 'pred': out}

        # compute loss
        if y is not None:
            # torch optimizes its computation internally and takes as input the logits instead
            loss = self.loss(out, y)
            result['loss'] = loss

        return result

    def loss(self, pred, y):
        return self.loss_fn(pred, y)

In [9]:
sent_classifier = SentencesClassifier(
    n_features=100, 
    n_hidden=64
)


In [10]:
class Trainer():
    def __init__(self, model, optimizer, device):

        self.device = device

        self.model = model
        self.optimizer = optimizer

        # starts requires_grad for all layers
        self.model.train()  # we are using this model for training (some layers have different behaviours in train and eval mode)
        self.model.to(self.device)  # move model to GPU if available

    def train(self, train_dataset, eval_dataset, epochs=1):

        train_loss = 0.0
        for epoch in tqdm(range(epochs)):
            epoch_loss = 0.0
            len_train = 0
            epoch_val_loss = 0.0
            len_val_train = 0
            accuracy = 0
            self.model.train()
            # each element (sample) in train_dataset is a batch
            for step, sample in enumerate(train_dataset):
                # inputs in the batch
                inputs = sample[0].to(self.device)
                # outputs in the batch
                targets = sample[1].to(self.device)
                output_distribution = self.model(inputs)
                loss = self.model.loss(output_distribution['pred'], targets)  # compute loss
                # calculates the gradient and accumulates
                loss.backward()  # we backpropagate the loss
                # updates the parameters
                self.optimizer.step()
                self.optimizer.zero_grad()

                epoch_loss += loss.item()
                len_train += 1
            
            self.model.eval()
            for step, sample in enumerate(eval_dataset):
                # inputs in the batch
                inputs = sample[0].to(self.device)
                # outputs in the batch
                targets = sample[1].to(self.device)
                output_distribution = self.model(inputs)
                loss = self.model.loss(output_distribution['pred'], targets)  # compute loss    
                
                accuracy += ((output_distribution['pred'] > 0.5) == targets).float().mean().item()
                epoch_val_loss += loss.item()
                len_val_train += 1
            
            avg_epoch_loss = epoch_loss / len_train
            avg_eval_loss = epoch_val_loss / len_val_train
            avg_accuracy_loss = accuracy / len_val_train
            print('Epoch: {} avg loss = {:0.4f} eval loss = {:0.4f} ACC = {}'.format(epoch, avg_epoch_loss, avg_eval_loss, avg_accuracy_loss))

            train_loss += avg_epoch_loss
            # torch.save(self.model.state_dict(),
            #            os.path.join(output_folder, 'state_{}.pt'.format(epoch)))  # save the model state

        avg_epoch_loss = train_loss / epochs
        return avg_epoch_loss

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = torch.optim.SGD(sent_classifier.parameters(), lr=0.02)
trainer = Trainer(sent_classifier, optimizer, device)
sentences_dm.setup('train')
train_dataloader = sentences_dm.test_dataloader()
avg_loss = trainer.train(train_dataloader,test_dataloader, epochs=300)